In [1]:
import os
os.chdir("../")
%pwd


'c:\\Users\\User\\Documents\\Kosi\\projects\\mlops\\hf\\text_summariser'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

In [2]:

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:Path
    local_data_file:Path
    unzip_dir:Path

In [3]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self, 
                 config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os 
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger

In [7]:
class DataIngestion:
    def __init__ (self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL,
                                                    filename=self.config.local_data_file)
            logger.info('Data is downloaded')
            
        else:
            logger.info("Data already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [11]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2024-12-17 01:24:04,570: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-17 01:24:04,573: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-17 01:24:04,575: INFO: common: created directory at: artifacts]
[2024-12-17 01:24:04,577: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-17 01:24:06,945: INFO: 1511048674: Data is downloaded]
